<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-04/RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From the data, it seems like measurements were taken during 3 different continuous sessions, which happen to be pretty good sizes for training, validation, and test sets.

In [11]:
import numpy as np
import pandas as pd
# import contractions
import re
import torch
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/YoutubeCommentsDataSet.csv")

# Used for balancing the dataset
def load_dataset(dat, tar):
  vals, cts = np.unique(tar, return_counts=True)
  min_val = vals[np.argmin(cts)]
  min_ct = min(cts)
  new_dat = dat[tar == min_val]
  new_tar = tar[tar == min_val]
  for i, v in enumerate(vals):
    if v != min_val:
      indices = list(np.where(tar == v))[0]
      new_ind = np.random.choice(indices, min_ct)
      new_dat = np.append(new_dat, dat[new_ind], axis=0)
      new_tar = np.append(new_tar, tar[new_ind])
  return new_dat, new_tar

  # Function to clean text for sentiment analysis
def clean_text(text):
    text = str(text).lower() # Convert to lowercase
    # text = contractions.fix(text) # Expand contractions
    text = re.sub(r"http\S+|www\.\S+", "", text) # Remove URLs starting with http/https or www
    text = re.sub(r"@\w+", "", text)  # Remove mentions (@usernames) to avoid irrelevant tokens
    text = re.sub(r"[^a-z0-9' ]", "", text) # Remove special characters
    text = re.sub(r"\s+", " ", text).strip() # Remove extra spaces
    return text

# Apply text cleaning function to the "Comment" column before model training
df["Cleaned_Text"] = df["Comment"].apply(clean_text)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
print(df.count(axis=0))
df.dropna(axis=0, inplace = True)
print(df.count(axis=0))
df = df.drop_duplicates(subset = "Comment")

Comment         18364
Sentiment       18408
Cleaned_Text    18408
dtype: int64
Comment         18364
Sentiment       18364
Cleaned_Text    18364
dtype: int64


In [13]:

ys = df['Sentiment']
df.pop('Sentiment')

,Sentiment
0,neutral
1,negative
2,positive
3,negative
4,positive
...,...
18403,positive
18404,positive
18405,neutral
18406,positive


In [14]:

dat = np.array(df)[:,0]
str2id = dict()
n_ids = 1
max_str = 0
for i in range(len(dat)):
  strs = dat[i].split()
  if len(strs) > max_str:
    max_str = len(strs)
  for s in strs:
    if s not in str2id.keys():
      str2id[s] = n_ids
      n_ids += 1

transformed_dat = np.zeros((np.shape(dat)[0], max_str), dtype = np.int32)
for i in range(len(dat)):
    strs = dat[i].split()
    if len(strs) > 200:
      continue
    for s in range(len(strs)):
      transformed_dat[i, s] = str2id[strs[s]]
max_str = 200

# print(transformed_y)

vals, idxs, occs = np.unique(ys, return_inverse = True, return_counts = True)
# print(vals, idxs, occs)

transformed_dat, ys = load_dataset(transformed_dat, np.array(ys))

transformed_y = np.zeros((np.shape(transformed_dat)[0], 3))
transformed_y[ys == 'positive'] = [1,0,0]
transformed_y[ys == 'neutral'] =  [0,1,0]
transformed_y[ys == 'negative'] = [0,0,1]

print(transformed_dat, transformed_y)



[[ 46   7  47 ...   0   0   0]
 [ 89  65  90 ...   0   0   0]
 [303 304  65 ...   0   0   0]
 ...
 [174 175 867 ...   0   0   0]
 [888 284  30 ...   0   0   0]
 [888 631 311 ...   0   0   0]] [[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [21]:
from torch import nn

class myRNN(nn.Module):
  def __init__(self, input_dim, embed_dim, hidden_dim, output_dim):
    super(myRNN, self).__init__()
    self.hidden_dim = hidden_dim
    self.embed = nn.Embedding(input_dim, embed_dim, padding_idx=0)
    # self.norm = nn.LayerNorm(embed_dim)
    self.rnn = nn.RNN(embed_dim, hidden_dim, nonlinearity='relu')
    self.final = nn.Linear(hidden_dim, output_dim)
    self.output = nn.Softmax(dim=1)


  def forward(self, x):
    batch_size = x.size()[0]
    em = self.embed(x)
    h0 = torch.zeros(1, batch_size, self.hidden_dim)
    res, _ = self.rnn(em, h0)
    final = self.final(res)[:,-1,:]
    return self.output(final)



class myLSTM(nn.Module):
  def __init__(self, input_dim, embed_dim, hidden_dim, output_dim):
    super().__init__()
    self.embed = nn.Embedding(input_dim, embed_dim)
    self.rnn = nn.LSTM(embed_dim, hidden_dim)
    self.final = nn.Linear(hidden_dim, output_dim)
    self.output = nn.Softmax(dim=1)


  def forward(self, x):
    em = self.embed(x)
    res, _ = self.rnn(em)
    final = self.final(res)[:,-1,:]
    return self.output(final)


class myGRU(nn.Module):

  def __init__(self, input_dim, embed_dim, hidden_dim, output_dim):
    super().__init__()
    self.batch = nn.BatchNorm1d(input_dim)
    self.embed = nn.Embedding(input_dim, embed_dim)
    self.rnn = nn.GRU(embed_dim, hidden_dim)
    self.final = nn.Linear(hidden_dim, output_dim)
    self.output = nn.Softmax(dim=1)


  def forward(self, x):
    b = self.batch(x)
    em = self.embed(b)
    res, _ = self.rnn(em)
    final = self.final(res)[:,-1,:]
    return self.output(final)



In [22]:
from torch import optim
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

indices = np.arange(np.shape(transformed_dat)[0])
np.random.shuffle(indices)
train_x = torch.tensor(transformed_dat[indices[:10]], dtype = torch.int)
train_y = torch.tensor(transformed_y[indices[:10]], dtype = torch.float)
val_x = torch.tensor(transformed_dat[indices[int(len(indices)*0.6):int(len(indices)*0.8)]], dtype = torch.int)
val_y = torch.tensor(transformed_y[indices[int(len(indices)*0.6):int(len(indices)*0.8)]], dtype = torch.float)
test_x = torch.tensor(transformed_dat[indices[int(len(indices)*0.8):]], dtype = torch.int)
test_y = torch.tensor(transformed_y[indices[int(len(indices)*0.8):]], dtype = torch.float)
print(train_x)

epochs = 30
batch_size = 32
val_epoch = 1

model = myRNN(n_ids, 256, 64, 3)
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

min_loss = 2
since_min = 0

for e in range(epochs):
  model.train()
  for batch in range(train_x.shape[0]//batch_size):
    pred = model(train_x[batch*batch_size:(batch+1)*batch_size])
    print(pred)
    loss = loss_fn(pred, train_y[batch*batch_size:(batch+1)*batch_size])
    if(batch%10 == 0):
      print("Epoch {}; training loss: {:.5f}".format(e, loss.item()))
    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 5)
    optimizer.step()

  if e % val_epoch == 0:
    model.eval()
    with torch.no_grad():
      pred = model(val_x)
      loss = loss_fn(pred, val_y)
      print("Validation loss: {:.5f}".format(loss.item()))
      print("Validation accuracy: {:.4f}".format(accuracy_score(np.argmax(val_y, axis=1), np.argmax(pred, axis=1))))
      if loss.item() < min_loss:
        min_loss = loss.item()
        since_min = 0
      else:
        since_min += 1
    if since_min >= 5:
      break

  print()


model.eval()
with torch.no_grad():
  pred = model(test_x)
  print(pred)
  pred = np.argmax(pred, axis=1)
  print(pred)
  print("\nTest metrics:")
  print("f1 score: {:.5f}".format(f1_score(np.argmax(test_y, axis=1), pred, average='macro')))
  print("recall score: {:.5f}".format(recall_score(np.argmax(test_y, axis=1), pred, average='macro')))
  print("precision score: {:.5f}".format(precision_score(np.argmax(test_y, axis=1), pred, average='macro')))
  print("Overall accuracy: {:.2f}%".format(accuracy_score(np.argmax(test_y, axis=1), pred)*100))

tensor([[ 1456,  1213,  6763,  ...,     0,     0,     0],
        [ 7797,    69,   133,  ...,     0,     0,     0],
        [38811,    65,  1599,  ...,     0,     0,     0],
        ...,
        [ 3505,    69,   133,  ...,     0,     0,     0],
        [  260,  2967,    65,  ...,     0,     0,     0],
        [  174,  1292,   170,  ...,     0,     0,     0]], dtype=torch.int32)


RuntimeError: Expected hidden size (1, 1353, 64), got [1, 1390, 64]

In [ ]:
from torch import optim
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

indices = np.arange(np.shape(transformed_dat)[0])
np.random.shuffle(indices)
train_x = torch.tensor(transformed_dat[indices[:int(len(indices)*0.1)]], dtype = torch.int)
train_y = torch.tensor(transformed_y[indices[:int(len(indices)*0.1)]], dtype = torch.float)
val_x = torch.tensor(transformed_dat[indices[int(len(indices)*0.6):int(len(indices)*0.8)]], dtype = torch.int)
val_y = torch.tensor(transformed_y[indices[int(len(indices)*0.6):int(len(indices)*0.8)]], dtype = torch.float)
test_x = torch.tensor(transformed_dat[indices[int(len(indices)*0.8):]], dtype = torch.int)
test_y = torch.tensor(transformed_y[indices[int(len(indices)*0.8):]], dtype = torch.float)

epochs = 20
val_epoch = 2

model = myLSTM(n_ids, 64, 64, 3)
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

min_loss = 2
since_min = 0

for e in range(epochs):
  model.train()

  pred = model(train_x)
  loss = loss_fn(pred, train_y)
  print("Epoch {}; training loss: {:.5f}".format(e, loss.item()))
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  if e % val_epoch == 0:
    model.eval()
    with torch.no_grad():
      pred = model(val_x)
      loss = loss_fn(pred, val_y)
      print("Validation loss: {:.5f}".format(loss.item()))
      if loss.item() < min_loss:
        min_loss = loss.item()
        since_min = 0
      else:
        since_min += 1
    if since_min >= 3:
      break
  print()


model.eval()
with torch.no_grad():
  pred = model(test_x)
  pred = np.argmax(pred, axis=1)
  print("\nTest metrics:")
  print("f1 score: {:.5f}".format(f1_score(np.argmax(test_y, axis=1), pred, average='macro')))
  print("recall score: {:.5f}".format(recall_score(np.argmax(test_y, axis=1), pred, average='macro')))
  print("precision score: {:.5f}".format(precision_score(np.argmax(test_y, axis=1), pred, average='macro')))
  print("Overall accuracy: {:.2f}%".format(accuracy_score(np.argmax(test_y, axis=1), pred)*100))